# Generate trimaps from U-2-Net alpha matte

This is the 2nd step in our workflow to remove the background from an image:

1. Use U-2-Net pre-trained model to generate a first alpha matte
2. **Use the U-2-Net alpha matte to generate a trimap** (the current colab notebook)
3. Train MODNet model with the original image, the trimap and ground truth image from DUTS dataset

## Sources:
* 


# Import

In [ ]:
# import modules to handle files
import os
import shutil
from google.colab import drive

# Mount Google Drive

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# Function

In [ ]:
def generate_trimap_from_alpha_matte(src_dir_path,
                                     dst_dir_path,
                                     open_size=10,
                                     alpha_margin=10):
  
  # if destination directory doesn't exists, create it
  if not os.path.exists(dst_dir_path):
    os.makedirs(dst_dir_path)

  # get list of alpha matte paths in source directory
  alpha_matte_path_list = glob.glob(src_dir_path + os.sep + '*')

  for alpha_matte_path in alpha_matte_path_list:

    # get alpha matte filename
    alpha_matte_name = alpha_matte_path.split(os.sep)[-1]

    # create destination path for the alpha matte
    dst_path = os.path.join(dst_dir, alpha_matte_name)

    # read alpha matte image into numpy array
    alpha_matte = cv2.imread(alpha_matte_path, 0)

    # make sure if the alpha_matte array has 2 dimensions 
    assert alpha_matte.ndim == 2

    # compute each region
    ## get foreground region
    foreground = ((255 - alpha_margin) < alpha_matte)

    ## get background region
    background = (alpha_matte < alpha_margin)

    ## get unknown region: outside of background and foreground
    unknown = ~(foreground + background)

    ## dilate unknown area with elliptical kernel.
    unknown = cv2.dilate(
        unknown.astype(np.uint8),
        cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_size, open_size))
    ).astype(np.bool)

    # draw trimap
    trimap = np.zeros_like(alpha_matte)
    trimap[foreground] = 255
    trimap[unknown] = 127

    # save trimap
    cv2.imwrite(dst_path, trimap)

# Generate trimaps for training and test sets

In [ ]:
# generate trimaps for training set
train_src_dir="/content/drive/MyDrive/Faktion/DUTS/DUTS-TR/DUTS-TR-u2net-MASK/"
train_dst_dir="/content/drive/MyDrive/Faktion/DUTS/DUTS-TR/DUTS-TR-Trimap/"
generate_trimap_from_alpha_matte(train_src_dir, train_dst_dir)

# generate trimaps for test set
test_src_dir="/content/drive/MyDrive/Faktion/DUTS/DUTS-TE/DUTS-TE-u2net-MASK/"
test_dst_dir="/content/drive/MyDrive/Faktion/DUTS/DUTS-TE/DUTS-TE-Trimap/"
generate_trimap_from_alpha_matte(test_src_dir, test_dst_dir)